In [69]:
using LinearAlgebra
using Plots
using Statistics

In [70]:
# Generate the Hankel, assuming z is a stacked vector with block dimension d
function Hankel(z::Array{Float64}, d::Int64, L::Int64)
    N = Int64(round(length(z) / d))
    @assert length(z) == N * d
    z = reshape(z, (d, N))
    H = Matrix{Float64}(zeros(d * L, N - L + 1))
    for i = 1:(N - L + 1); H[:, i] = (z[:, i:(i + L - 1)])[:] end
    return H
end
;

In [71]:
# For a particular value of N, calculate the statistics of averaging N different Hw's
# Generate M (=1000) different norm(Hw) to get the mean, delta, and (1-delta) quantiles
function sampleTrueN(n::Int64, sigma::Float64, T::Int64, L::Int64, N::Int64, delta::Float64; M=1000)
    norms = zeros(M)
    for m = 1:M
        w = (sigma / sqrt(N)) * randn(n * T)
        Hw = Hankel(w, n, L)
        norms[m] = opnorm(Hw)
    end
    # Mean
    mu = mean(norms)
    
    # delta quantile
    qlonorm = quantile(norms, delta)
    
    # (1-delta) quantile
    qhinorm = quantile(norms, 1 - delta)
    
    return (mu, qlonorm, qhinorm)
end


# For a particular N, calculate the statistics of 
# Run a bootstrap experiment:
# Over M (=1) iterations, get call sampleTrueN to figure out the bootstrap epsilon
# The bootstrap epsilon is the (1-delta) quantile of the empirical true H_L (w)
function sampleBootN(n::Int64, sigma::Float64, T::Int64, L::Int64, N::Int64, delta::Float64; M=1)
    Bs = zeros(M)
    for m = 1:M
        W = sigma * randn((n * T), N)
        (mu, qlo, qhi) = sampleTrueN(n, sigma, T, L, N, delta)
        Bs[m] = qhi
    end
    
    # Mean of boot eps
    muB = mean(Bs)
    
    # delta quantile of boot eps
    qloB = quantile(Bs, delta)
    
    # (1-delta) quantile of boot eps
    qhiB = quantile(Bs, 1 - delta)
    
    return (muB, qloB, qhiB)
end

;

In [18]:
# Slightly unstable graph Laplacian is dim n = 3
n = 3
sigma = sqrt(0.1)

# Hankel matrix dimensions
T = 45
L = 10

# Uncertainty parameter delta
delta = 0.05

# 10, 20, 30, 40, ..., 6000
Ns = 10:10:6000

println("Generating trues")
# About 60 secs
@time trues = [sampleTrueN(n, sigma, T, L, k, delta) for k in Ns]

println("Generating boots")
# About 60 secs
@time boots = [sampleBootN(n, sigma, T, L, k, delta) for k in Ns]
;

Generating trues
 54.547918 seconds (51.65 M allocations: 40.982 GiB, 3.10% gc time)
Generating boots
 54.874407 seconds (51.65 M allocations: 44.610 GiB, 3.50% gc time)


600-element Array{Tuple{Float64,Float64,Float64},1}:
 (1.2451208309269783, 1.2451208309269783, 1.2451208309269783)      
 (0.8800208738612182, 0.8800208738612182, 0.8800208738612182)      
 (0.7200425738094807, 0.7200425738094807, 0.7200425738094807)      
 (0.6221722052666475, 0.6221722052666475, 0.6221722052666475)      
 (0.5588345628901507, 0.5588345628901507, 0.5588345628901507)      
 (0.5134833357379155, 0.5134833357379155, 0.5134833357379155)      
 (0.4725056678627814, 0.4725056678627814, 0.4725056678627814)      
 (0.4391189684675182, 0.4391189684675182, 0.4391189684675182)      
 (0.4172465501444145, 0.4172465501444145, 0.4172465501444145)      
 (0.3974732989345227, 0.3974732989345227, 0.3974732989345227)      
 (0.37960548382009573, 0.37960548382009573, 0.37960548382009573)   
 (0.35767257836314964, 0.35767257836314964, 0.35767257836314964)   
 (0.3472707612336057, 0.3472707612336057, 0.3472707612336057)      
 ⋮                                                             

In [50]:
NN = length(Ns)
# Mean of true ||Hw||
muTs = [trues[i][1] for i = 1:NN]

# delta quantile of true ||Hw||
qloTs = [trues[i][2] for i = 1:NN]

# (1-delta) quantile of true ||Hw||
qhiTs = [trues[i][3] for i = 1:NN]

# Mean of boot eps
muBs = [boots[i][1] for i = 1:NN]

# We don't actually plot these two
qloBs = [boots[i][2] for i = 1:NN]
qhiBs = [boots[i][3] for i = 1:NN]
;

In [74]:
# Plot the true ||Hw|| data, along with ribbons
plot(Ns, muTs,
    ribbon=(muTs - qloTs, qhiTs - muTs),
    fillalpha=0.1,
    label="true ||H_L (w)||",
    xlabel="Number of Samples (N)",
    ylabel="Operator Norm",
    yscale=:log10,
    xtickfontsize=18,
    ytickfontsize=18,
    legendfontsize=18,
    xguidefontsize=18,
    yguidefontsize=18
)


# Plot the solid eps-B line
plot!(Ns, muBs,
    label="ε Boot"
)

# Check which directory you booted jupyter notebook from
savefig("Hwnorms.png")


In [76]:
[
    muBs[1];
    muBs[2];
    muBs[4];
    muBs[8];
    muBs[16];
    muBs[32];
    muBs[64];
    muBs[128];
    muBs[256];
    muBs[512];
]

10-element Array{Float64,1}:
 1.2451208309269783  
 0.8800208738612182  
 0.6221722052666475  
 0.4391189684675182  
 0.3126961297989887  
 0.2238407593985623  
 0.15557654473097288 
 0.110852519282565   
 0.07787090540704344 
 0.055482199975597016